# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [21]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head(15)

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
5,7.4,0.660,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,0
6,7.9,0.600,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,0
7,7.3,0.650,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,1
8,7.8,0.580,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,1
9,7.5,0.500,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column

y = loans_df["credit_ranking"]

# Display a sample of y
y

0       0
1       0
2       0
3       1
4       0
       ..
1594    0
1595    1
1596    1
1597    0
1598    1
Name: credit_ranking, Length: 1599, dtype: int64

In [6]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop("credit_ranking", axis=1)

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)


X_train_scaled = X_scaler.transform(X_train)      # scale the training data
X_test_scaled = X_scaler.transform(X_test)        # scale the testing data

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
input_nodes

11

In [10]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_L1 = 6

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_L2 = 3

# Define the number of neurons in the output layer
neurons = 1

In [11]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_L1, activation="relu", input_dim=input_nodes))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_L2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=neurons, activation="sigmoid"))

In [12]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 3)                 21        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [13]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)  # Need to use the scaled training data containing the features

Epoch 1/50
38/38 [==============================] - 4s 9ms/step - loss: 0.7400 - accuracy: 0.4487
Epoch 2/50
38/38 [==============================] - 0s 7ms/step - loss: 0.7068 - accuracy: 0.4737
Epoch 3/50
38/38 [==============================] - 0s 7ms/step - loss: 0.6913 - accuracy: 0.5371
Epoch 4/50
38/38 [==============================] - 0s 11ms/step - loss: 0.6824 - accuracy: 0.5947
Epoch 5/50
38/38 [==============================] - 0s 10ms/step - loss: 0.6761 - accuracy: 0.6305
Epoch 6/50
38/38 [==============================] - 0s 12ms/step - loss: 0.6702 - accuracy: 0.6572
Epoch 7/50
38/38 [==============================] - 1s 13ms/step - loss: 0.6643 - accuracy: 0.6789
Epoch 8/50
38/38 [==============================] - 0s 7ms/step - loss: 0.6579 - accuracy: 0.6839
Epoch 9/50
38/38 [==============================] - 0s 10ms/step - loss: 0.6511 - accuracy: 0.6981
Epoch 10/50
38/38 [==============================] - 0s 8ms/step - loss: 0.6435 - accuracy: 0.7023
Epoch 11/50
38

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [15]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)


# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 0.5468 - accuracy: 0.7200 - 174ms/epoch - 13ms/step
Loss: 0.5467822551727295, Accuracy: 0.7200000286102295


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [16]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Export your model to a keras file
nn_model.save(file_path)    # save function performs the write/export

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [17]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [22]:
# Make predictions with the test data
predictions = nn_imported.predict(X_test_scaled,verbose=2)

# Display a sample of the predictions
predictions

13/13 - 0s - 90ms/epoch - 7ms/step


array([[0.26252675],
       [0.45627275],
       [0.74219894],
       [0.74219894],
       [0.74219894],
       [0.74219894],
       [0.74219894],
       [0.375255  ],
       [0.6152755 ],
       [0.35315397],
       [0.74219894],
       [0.18500578],
       [0.46110037],
       [0.74219894],
       [0.49046168],
       [0.47044092],
       [0.74219894],
       [0.44153422],
       [0.5372582 ],
       [0.32854375],
       [0.6452237 ],
       [0.74219894],
       [0.31301233],
       [0.74219894],
       [0.5535183 ],
       [0.74219894],
       [0.7273934 ],
       [0.56265986],
       [0.6390306 ],
       [0.31336656],
       [0.598009  ],
       [0.74219894],
       [0.2603864 ],
       [0.74219894],
       [0.17938095],
       [0.5081898 ],
       [0.22270478],
       [0.4197704 ],
       [0.74219894],
       [0.17893551],
       [0.74219894],
       [0.17510293],
       [0.07416668],
       [0.74219894],
       [0.12877023],
       [0.5678004 ],
       [0.14671488],
       [0.355

In [23]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_rounded = [round(prediction[0],0) for prediction in predictions]       # round the prediction to the nearest integer - 0 decimal places
rounded_predictions_df = pd.DataFrame(predictions_rounded, columns=['prediction'])  # set the column in the new dataframe to "prediction"
rounded_predictions_df

,prediction
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [24]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, rounded_predictions_df))

              precision    recall  f1-score   support

           0       0.70      0.71      0.71       188
           1       0.74      0.73      0.73       212

    accuracy                           0.72       400
   macro avg       0.72      0.72      0.72       400
weighted avg       0.72      0.72      0.72       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**  
Current income (if any): This can be used to consider amount of loan in conjunction with saved money (501 plans, etc), for loan repayment.
Prior Payment History (if any):  History on prior loans to ascertain credit risk.
501 Plan balances (if any):  This can be used to gauge ability to repay the loan.
Major of Study and GPA:  Used to ascertain amount to loan and ability to repay.
Any scholarship funds or VA benefits available:  Same as 501 Plan and Major of Study & GPA. 



**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**
Content-based filtering because the data is specific to the student applicant. No other outside influences from others would come to play.  Based on the applicant's specific responses, this would drive the decisions and course that the model would take in making recommendations to such applicant.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**
1) Prior Loan Payment History:  If delinquent on any prior loans, this would raise a flag of course on credit risk
                                If no prior history at all, would need backing of a co-signer such as parent, etc.
2) Selection of study for Major and GPA:  If course of study is in an area without a sufficient income to support loan re-payment and/or GPA is low, this too would require a co-signer.  Of course if 529 balances and/or VA benefits or similar funds such as scholarship funds exist, a co-signer may not be required.   


